In [1]:
from xno.data.datasets.hdf5_dataset import H5pyDataset
from utils import MatReader
from torch.utils.data import DataLoader, TensorDataset, Dataset, default_collate

In [2]:
from data_loader import _1d_burger, _2d_ionize


In [2]:
import torch
import matplotlib.pyplot as plt
import sys
from xno.models import XNO
from xno.data.datasets import load_darcy_flow_small
from xno.utils import count_model_params
from xno.training import AdamW
from xno.training.incremental import IncrementalFNOTrainer
from xno.data.transforms.data_processors import IncrementalDataProcessor
from xno import LpLoss, H1Loss

In [3]:
data_path="/Users/sina/Documents/GitHub_Local/XNO/use_cases/data/1D_Lorenz_rho10.mat"

In [4]:
s = 2048
batch_size_train = 16
batch_size_vali = 16
batch_size_test = 16

In [5]:
reader = MatReader(data_path)
x_train = reader.read_field('f_train')
y_train = reader.read_field('u_train')
grid_x_train = reader.read_field('x_train')

x_vali = reader.read_field('f_vali')
y_vali = reader.read_field('u_vali')
grid_x_vali = reader.read_field('x_vali')

x_test = reader.read_field('f_test')
y_test = reader.read_field('u_test')
grid_x_test = reader.read_field('x_test') 

x_train = x_train.reshape(x_train.shape[0],s,1)
x_vali = x_vali.reshape(x_vali.shape[0],s,1)
x_test = x_test.reshape(x_test.shape[0],s,1)


In [6]:
# Define the custom Dataset
class DictDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return {'x': self.x[idx], 'y': self.y[idx]}

In [7]:
x_train.shape, y_train.shape

(torch.Size([200, 2048, 1]), torch.Size([200, 2048]))

In [8]:
x_vali.shape, y_vali.shape

(torch.Size([50, 2048, 1]), torch.Size([50, 2048]))

In [9]:
x_train = x_train.permute(0, 2, 1)
y_train = y_train.unsqueeze(1)
x_vali = x_vali.permute(0, 2, 1)
y_vali = y_vali.unsqueeze(1)

In [10]:
train_loader = DictDataset(x_train, y_train)
test_loader = DictDataset(x_vali, y_vali)

In [11]:
train_loader = DataLoader(train_loader, batch_size=batch_size_train, shuffle=True)
test_loader = DataLoader(test_loader, batch_size=batch_size_vali, shuffle=True)
test_loader = {
    2048: test_loader
}

In [12]:
batch = next(iter(train_loader))
type(train_loader), type(batch), batch['x'].shape, batch['y'].shape

(torch.utils.data.dataloader.DataLoader,
 dict,
 torch.Size([16, 1, 2048]),
 torch.Size([16, 1, 2048]))

In [13]:
batch = next(iter(test_loader[2048]))
type(test_loader), type(batch), batch['x'].shape, batch['y'].shape

(dict, dict, torch.Size([16, 1, 2048]), torch.Size([16, 1, 2048]))

In [14]:
len(test_loader[2048])

4

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
model = XNO(
    max_n_modes=(16, ),
    n_modes=(2, ),
    hidden_channels=32,
    in_channels=1,
    out_channels=1,
    transformation="hno",
    # transformation_kwargs={"wavelet_level": 2, "wavelet_size": [2048]}
)
model = model.to(device)
n_params = count_model_params(model)

======== Selected Kernel Description =======
Dimentionality: 1D
Transformation: [ Hilbert Neural Operator (HNO) Kernel ]
>>> Overview:
The HNO applies Hilbert Transform, emphasizing the phase-shifted features of the input
signal for enhanced data representation.

>>> Key Features:
- Focuses on phase information, useful in signal processing.
- Suitable for scenarios requiring advanced spectral analysis.

>>> Reference:
This is an experimental implementation. Currently no formal reference.

================== Config ==================
>>> Normaliztion: None
>>> Activation Function: 



In [17]:
optimizer = AdamW(model.parameters(), lr=8e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

In [18]:
data_transform = IncrementalDataProcessor(
    in_normalizer=None,
    out_normalizer=None,
    device=device,
    subsampling_rates=[2, 1],
    dataset_resolution=2048,
    dataset_indices=[2],
    epoch_gap=10,
    verbose=True,
)

data_transform = data_transform.to(device)

Original Incre Res: change index to 0
Original Incre Res: change sub to 2
Original Incre Res: change res to 1024


In [19]:
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)
train_loss = h1loss
eval_losses = {"h1": h1loss, "l2": l2loss}
print("\n### N PARAMS ###\n", n_params)
print("\n### OPTIMIZER ###\n", optimizer)
print("\n### SCHEDULER ###\n", scheduler)
print("\n### LOSSES ###")
print("\n### INCREMENTAL RESOLUTION + GRADIENT EXPLAINED ###")
print(f"\n * Train: {train_loss}")
print(f"\n * Test: {eval_losses}")
sys.stdout.flush()


### N PARAMS ###
 144161

### OPTIMIZER ###
 AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 0.008
    lr: 0.008
    weight_decay: 0.0001
)

### SCHEDULER ###

### LOSSES ###

### INCREMENTAL RESOLUTION + GRADIENT EXPLAINED ###

 * Train: <xno.losses.data_losses.H1Loss object at 0x17fa32830>

 * Test: {'h1': <xno.losses.data_losses.H1Loss object at 0x17fa32830>, 'l2': <xno.losses.data_losses.LpLoss object at 0x308a89d80>}


In [20]:
# Finally pass all of these to the Trainer
trainer = IncrementalFNOTrainer(
    model=model,
    n_epochs=10,
    data_processor=data_transform,
    device=device,
    verbose=True,
    incremental_loss_gap=False,
    incremental_grad=True,
    incremental_grad_eps=0.9999,
    incremental_loss_eps = 0.001,
    incremental_buffer=5,
    incremental_max_iter=1,
    incremental_grad_max_iter=2,
)

In [21]:
trainer.train(
    train_loader,
    test_loader,
    optimizer,
    scheduler,
    regularizer=False,
    training_loss=train_loss,
    eval_losses=eval_losses,
    save_every=1,
    save_testing=True,
    save_dir="save/"
)

Training on 200 samples
Testing on [50] samples         on resolutions [2048].
Raw outputs of shape torch.Size([16, 1, 1024])
[0] time=4.57, avg_loss=0.9567, train_err=14.7188
True
Eval: 2048_h1=0.9682, 2048_l2=0.9679
[Rank 0]: saved training state to save/
[1] time=4.59, avg_loss=0.8856, train_err=13.6241
True
Eval: 2048_h1=0.7736, 2048_l2=0.8846
[Rank 0]: saved training state to save/
[2] time=4.97, avg_loss=0.8423, train_err=12.9584
True
Eval: 2048_h1=0.7754, 2048_l2=0.8605
[Rank 0]: saved training state to save/
[3] time=4.80, avg_loss=0.8157, train_err=12.5485
True
Eval: 2048_h1=0.7636, 2048_l2=0.8665
[Rank 0]: saved training state to save/
[4] time=4.59, avg_loss=0.7951, train_err=12.2325
True
Eval: 2048_h1=0.7335, 2048_l2=0.8465
[Rank 0]: saved training state to save/
[5] time=4.63, avg_loss=0.7857, train_err=12.0871
True
Eval: 2048_h1=0.7415, 2048_l2=0.8596
[Rank 0]: saved training state to save/
[6] time=4.65, avg_loss=0.7750, train_err=11.9233
True
Eval: 2048_h1=0.7390, 2048_

{'train_err': 11.62308615904588,
 'avg_loss': 0.7555006003379822,
 'avg_lasso_loss': None,
 'epoch_train_time': 4.689389250001113,
 '2048_h1': tensor(0.7585),
 '2048_l2': tensor(1.0215)}

In [26]:
file_path = "save/test_results.pt"  # or Path(...)
data = torch.load(file_path)

/var/folders/94/p22yrw514tv4kplhpq9hmknr0000gn/T/ipykernel_61588/3952325940.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)


In [31]:
type(data['pred']), data['pred'].shape

(torch.Tensor, torch.Size([50, 1, 2048]))

In [29]:
import torch
from scipy.io import savemat

# Load the .pt file
pt_file_path = "save/test_results.pt"
data = torch.load(pt_file_path)

# Convert tensors to NumPy arrays (required for .mat format)
x_array = data["x"].cpu().numpy()
y_array = data["y"].cpu().numpy()
pred_array = data["pred"].cpu().numpy()

# Save to .mat
mat_file_path = "test_results.mat"
savemat(mat_file_path, {"x": x_array, "y": y_array, "pred": pred_array})

print(f"Converted {pt_file_path} to {mat_file_path}")


Converted save/test_results.pt to test_results.mat


/var/folders/94/p22yrw514tv4kplhpq9hmknr0000gn/T/ipykernel_61588/3978367509.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(pt_file_path)
